# Experiment 006: Grid-Based Initialization for Worst N Values

Implement zaburo-style grid initialization for worst-performing N values.
The key insight: trees in alternating rows (0° and 180°) interlock well.

**Worst N values to target:** N=1, 49, 21, 50, 20, 37, 53, 51, 34, 19

In [1]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree
import os
import shutil

getcontext().prec = 25
scale_factor = Decimal('1e15')

print("Libraries loaded")

Libraries loaded


In [2]:
# ChristmasTree class
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        trunk_w, trunk_h = Decimal('0.15'), Decimal('0.2')
        base_w, mid_w, top_w = Decimal('0.7'), Decimal('0.4'), Decimal('0.25')
        tip_y, tier_1_y, tier_2_y, base_y = Decimal('0.8'), Decimal('0.5'), Decimal('0.25'), Decimal('0.0')
        trunk_bottom_y = -trunk_h
        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x * scale_factor), yoff=float(self.center_y * scale_factor))

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
# Zaburo-style grid initialization
def find_best_grid_trees(n):
    """Find best grid-based arrangement for n trees."""
    best_score, best_trees = float("inf"), None
    
    for n_even in range(1, n + 1):
        for n_odd in [n_even, n_even - 1]:
            if n_odd < 0:
                continue
            all_trees = []
            rest = n
            r = 0
            while rest > 0:
                m = min(rest, n_even if r % 2 == 0 else n_odd)
                if m <= 0:
                    break
                rest -= m
                
                angle = 0 if r % 2 == 0 else 180
                x_offset = Decimal('0') if r % 2 == 0 else Decimal('0.7') / 2
                y = Decimal(str(r // 2)) * Decimal('1.0') if r % 2 == 0 else (Decimal('0.8') + Decimal(str((r - 1) // 2)) * Decimal('1.0'))
                
                row_trees = [ChristmasTree(
                    center_x=str(Decimal('0.7') * i + x_offset),
                    center_y=str(y),
                    angle=str(angle)
                ) for i in range(m)]
                all_trees.extend(row_trees)
                r += 1
            
            if len(all_trees) != n:
                continue
                
            # Calculate score
            xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in all_trees])
            min_x, min_y = xys.min(axis=0)
            max_x, max_y = xys.max(axis=0)
            side = max(max_x - min_x, max_y - min_y)
            score = side ** 2 / n
            
            if score < best_score:
                best_score = score
                best_trees = all_trees
    
    return best_score, best_trees

print("Grid initialization function defined")

Grid initialization function defined


In [4]:
# Fast scoring functions
TX = np.array([0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125])
TY = np.array([0.8,0.5,0.5,0.25,0.25,0,0,-0.2,-0.2,0,0,0.25,0.25,0.5,0.5])

def score_group_fast(xs, ys, degs):
    n = len(xs)
    if n == 0:
        return float('inf')
    all_x, all_y = [], []
    for i in range(n):
        rad = np.radians(degs[i])
        c, s = np.cos(rad), np.sin(rad)
        px = TX * c - TY * s + xs[i]
        py = TX * s + TY * c + ys[i]
        all_x.extend(px)
        all_y.extend(py)
    all_x, all_y = np.array(all_x), np.array(all_y)
    side = max(all_x.max() - all_x.min(), all_y.max() - all_y.min())
    return side * side / n

def strip_s(val):
    s = str(val)
    return float(s[1:] if s.startswith('s') else s)

def get_side(trees):
    if not trees:
        return 0.0
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in trees])
    return max(xys.max(axis=0) - xys.min(axis=0))

def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        for idx in tree_index.query(poly):
            if idx != i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

print("Scoring functions defined")

Scoring functions defined


In [5]:
# Load current best submission
df = pd.read_csv('/home/code/experiments/005_extended_optimization/submission_fd.csv')

# Calculate current scores per N
current_scores = {}
for n in range(1, 201):
    group = df[df['id'].str.startswith(f'{n:03d}_')]
    if len(group) == n:
        xs = group['x'].apply(strip_s).values
        ys = group['y'].apply(strip_s).values
        degs = group['deg'].apply(strip_s).values
        current_scores[n] = score_group_fast(xs, ys, degs)

current_total = sum(current_scores.values())
print(f"Current total score: {current_total:.6f}")

# Find worst N values
worst_n = sorted(current_scores.items(), key=lambda x: -x[1])[:20]
print("\nTop 20 worst N values:")
for n, score in worst_n:
    print(f"  N={n}: {score:.6f}")

Current total score: 84.712432

Top 20 worst N values:
  N=1: 0.661250
  N=49: 0.490000
  N=21: 0.487619
  N=50: 0.480200
  N=20: 0.478371
  N=37: 0.476757
  N=53: 0.471698
  N=51: 0.470784
  N=34: 0.470588
  N=19: 0.468739
  N=32: 0.468214
  N=76: 0.465822
  N=22: 0.465455
  N=38: 0.464211
  N=54: 0.462963
  N=52: 0.461731
  N=45: 0.460056
  N=18: 0.460021
  N=77: 0.459773
  N=35: 0.457143


In [ ]:
# Try grid initialization for worst N values and compare
print("\nComparing grid initialization vs current for worst N values:")
print("=" * 60)

improvements = []
for n, current_score in worst_n:
    grid_score, grid_trees = find_best_grid_trees(n)
    
    # Check for overlaps in grid solution
    if grid_trees and not has_overlap(grid_trees):
        improvement = current_score - grid_score
        if improvement > 0:
            improvements.append((n, current_score, grid_score, improvement, grid_trees))
            print(f"  N={n}: Current={current_score:.6f}, Grid={grid_score:.6f}, Improvement={improvement:.6f} ✓")
        else:
            print(f"  N={n}: Current={current_score:.6f}, Grid={grid_score:.6f}, No improvement")
    else:
        print(f"  N={n}: Grid solution has overlaps or failed")

print(f"\nFound {len(improvements)} improvements")

In [ ]:
# Apply improvements to submission
if improvements:
    print("\nApplying improvements...")
    
    # Load current submission
    df = pd.read_csv('/home/code/experiments/005_extended_optimization/submission_fd.csv')
    
    total_improvement = 0
    for n, old_score, new_score, improvement, grid_trees in improvements:
        # Remove old rows for this N
        df = df[~df['id'].str.startswith(f'{n:03d}_')]
        
        # Add new rows
        new_rows = []
        for i, tree in enumerate(grid_trees):
            new_rows.append({
                'id': f'{n:03d}_{i}',
                'x': f's{float(tree.center_x)}',
                'y': f's{float(tree.center_y)}',
                'deg': f's{float(tree.angle)}'
            })
        df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
        total_improvement += improvement
        print(f"  Replaced N={n}: {old_score:.6f} -> {new_score:.6f}")
    
    # Sort by id
    df['n'] = df['id'].str.split('_').str[0].astype(int)
    df['i'] = df['id'].str.split('_').str[1].astype(int)
    df = df.sort_values(['n', 'i']).drop(columns=['n', 'i'])
    
    # Save
    df.to_csv('/home/code/experiments/006_grid_initialization/submission_grid.csv', index=False)
    print(f"\nTotal improvement: {total_improvement:.6f}")
else:
    print("No improvements found - copying current best")
    shutil.copy('/home/code/experiments/005_extended_optimization/submission_fd.csv',
                '/home/code/experiments/006_grid_initialization/submission_grid.csv')

In [ ]:
# Validate and score the new submission
df_new = pd.read_csv('/home/code/experiments/006_grid_initialization/submission_grid.csv')

new_total = 0
overlaps = []
for n in range(1, 201):
    group = df_new[df_new['id'].str.startswith(f'{n:03d}_')]
    if len(group) == n:
        xs = group['x'].apply(strip_s).values
        ys = group['y'].apply(strip_s).values
        degs = group['deg'].apply(strip_s).values
        new_total += score_group_fast(xs, ys, degs)
        
        # Check overlaps using Shapely
        trees = [ChristmasTree(str(xs[i]), str(ys[i]), str(degs[i])) for i in range(n)]
        if has_overlap(trees):
            overlaps.append(n)

print(f"New total score: {new_total:.6f}")
print(f"Overlapping groups: {overlaps}")
print(f"Validation: {'PASSED' if len(overlaps) == 0 else 'FAILED'}")

In [ ]:
# Copy to submission folder if valid
if len(overlaps) == 0:
    shutil.copy('/home/code/experiments/006_grid_initialization/submission_grid.csv',
                '/home/submission/submission.csv')
    print("Copied to /home/submission/submission.csv")

print(f"\n=== SUMMARY ===")
print(f"Previous best: {current_total:.6f}")
print(f"New score: {new_total:.6f}")
print(f"Improvement: {current_total - new_total:.6f}")
print(f"Target: 68.931058")
print(f"Gap to target: {new_total - 68.931058:.6f}")